In [1]:
#import from the bronze layer

In [2]:
import os
import pandas as pd

BASE_PATH = "E:/C-DAC/Major Project/AI-Based-Maritime-Port-Intelligence"

BRONZE_PATH = os.path.join(BASE_PATH, "data", "bronze")
SILVER_PATH = os.path.join(BASE_PATH, "data", "silver")

os.makedirs(SILVER_PATH, exist_ok=True)

df1 = pd.read_csv(os.path.join(BRONZE_PATH, "ais_kattegat.csv"))
df2 = pd.read_csv(os.path.join(BRONZE_PATH, "ais_global.csv"))
df3 = pd.read_csv(os.path.join(BRONZE_PATH, "ais_unacorn.csv"))
df4 = pd.read_csv(os.path.join(BRONZE_PATH,"container_tracking.csv"))


C:\Users\harsh\AppData\Local\Temp\ipykernel_21984\3868569386.py:14: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv(os.path.join(BRONZE_PATH,"container_tracking.csv"))


# Silver

In [3]:
#Make new columns for standrdizing 

In [4]:
AIS_COLUMN_MAP = {
    "MMSI": "vessel_id",
    "mmsi": "vessel_id",
    "BaseDateTime": "event_time",
    "date": "event_time",
    "LAT": "lat",
    "LON": "lon",
    "SOG": "sog",
    "COG": "cog",
    "Heading": "heading",
    "VesselType": "vessel_type",
    "Length": "length",
    "Width": "width",
    "Draft": "draft"
}


In [5]:
#using Function change columns name of df1,df2,df3 

In [6]:
def standardize_columns(df, column_map):
    return df.rename(columns={k: v for k, v in column_map.items() if k in df.columns})

df1 = standardize_columns(df1, AIS_COLUMN_MAP)
df2 = standardize_columns(df2, AIS_COLUMN_MAP)
df3 = standardize_columns(df3, AIS_COLUMN_MAP)


In [7]:
#Add Metadata Columns (BECAUSE OF THIS WE UNDERSTAND WHICH DATA SET WHICH LINE IS GETTING)

In [8]:
df1["data_source"] = "kattegat"
df2["data_source"] = "global"
df3["data_source"] = "unacorn"

In [9]:
#Convert Time Columns to Datetime

In [10]:
for df in [df1, df2, df3]:
    if "event_time" in df.columns:
        df["event_time"] = pd.to_datetime(df["event_time"], errors="coerce")


In [11]:
#For container tracking (df4)

In [12]:
time_cols = [
    "updated", "etaSchedule", "eta", "ata",
    "etdSchedule", "etd", "atd"
]

for col in time_cols:
    if col in df4.columns:
        df4[col] = pd.to_datetime(df4[col], errors="coerce")


In [13]:
#Remove Duplicates

In [14]:
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()
df3 = df3.drop_duplicates()
df4 = df4.drop_duplicates()

In [15]:
# There is NO DUPLICATES VALUE IN DATA 

In [16]:
#Keep Only Useful Columns (REMOVE COLUMNS UNWANTED)

In [17]:
AIS_KEEP_COLS = [
    "vessel_id", "event_time", "lat", "lon",
    "sog", "cog", "heading",
    "vessel_type", "length", "width", "draft",
    "data_source"
]

df1 = df1[[c for c in AIS_KEEP_COLS if c in df1.columns]]
df2 = df2[[c for c in AIS_KEEP_COLS if c in df2.columns]]
df3 = df3[[c for c in AIS_KEEP_COLS if c in df3.columns]]

In [18]:
#Combine AIS Datasets

In [19]:
ais_cleaned = pd.concat([df1, df2, df3], ignore_index=True)

In [20]:
ais_cleaned.info()
ais_cleaned.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7629254 entries, 0 to 7629253
Data columns (total 12 columns):
 #   Column       Dtype         
---  ------       -----         
 0   vessel_id    int64         
 1   sog          float64       
 2   cog          float64       
 3   heading      float64       
 4   length       float64       
 5   width        float64       
 6   data_source  object        
 7   event_time   datetime64[ns]
 8   lat          float64       
 9   lon          float64       
 10  vessel_type  float64       
 11  draft        float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 698.5+ MB


,vessel_id,sog,cog,heading,length,width,data_source,event_time,lat,lon,vessel_type,draft
7629249,503074050,0.0,-1.0,-1.0,13.0,NaN,unacorn,2020-12-10,NaN,NaN,NaN,NaN
7629250,209138000,17.5,66.6,67.0,180.0,NaN,unacorn,2020-12-10,NaN,NaN,NaN,NaN
7629251,636015005,14.1,341.8,341.0,155.0,NaN,unacorn,2020-12-10,NaN,NaN,NaN,NaN
7629252,305057000,12.0,333.4,337.0,138.0,NaN,unacorn,2020-12-10,NaN,NaN,NaN,NaN
7629253,520317000,8.4,158.2,158.2,16.0,NaN,unacorn,2020-12-10,NaN,NaN,NaN,NaN


In [21]:
#Clean Container Tracking Dataset(df4)

In [22]:
CONTAINER_KEEP_COLS = [
    "ship", "imo",
    "lat", "long", "sog", "cog", "hdg",
    "depPort", "arrPort",
    "etaSchedule", "eta", "ata",
    "etdSchedule", "etd", "atd"
]

df4_cleaned = df4[[c for c in CONTAINER_KEEP_COLS if c in df4.columns]]


In [23]:
df4_cleaned = df4_cleaned.rename(columns={
    "long": "lon",
    "hdg": "heading"
})


In [24]:
df4_cleaned.head()

,ship,imo,lat,lon,sog,cog,heading,depPort,arrPort,etaSchedule,eta,ata,etdSchedule,etd,atd
0,Megastar,9773064,60.1469,24.9135,6.3,219,216,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:23:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20
1,Megastar,9773064,60.1445,24.9100,11.6,217,217,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:29:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20
2,Megastar,9773064,60.1412,24.9061,14.2,198,199,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:30:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20
3,Star,9364722,59.4462,24.7726,3.7,17,159,EETLL,FIHEL,2018-05-04 21:30:00,2018-04-05 21:26:00,2018-04-05 21:46:00,2018-05-04 19:30:00,2018-07-04 15:25:00,2018-04-05 19:21:17
4,Megastar,9773064,60.1344,24.9056,15.9,179,179,FIHEL,EETLL,2018-05-04 21:30:00,2018-04-05 21:25:00,2018-04-05 21:32:00,2018-05-04 19:30:00,2018-07-04 15:29:00,2018-04-05 19:18:20


In [25]:
# 1 ais_cleaned combination of (df1,df2,df3)
# 2 df4_cleaned (df4 data set )

In [26]:
ais_cleaned.to_csv(os.path.join(SILVER_PATH, "ais_cleaned.csv"), index=False)
df4_cleaned.to_csv(os.path.join(SILVER_PATH, "container_tracking_cleaned.csv"), index=False)